In [1]:
#!pip install pyshp
#!pip install pyproj

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

###2. Data 읽기

In [3]:
#file 읽기 (encoding 주의)
light_dataframe = pd.read_csv('data/전국보안등정보표준데이터.csv', encoding='EUC-KR')

C:\Users\ARA\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


####3_2. 지역필터링
1. 원하는 지역 필터를 위한 열로  결측치가 많은 '소재지도로명주소', '소재지지번주소'는 부적합하므로 제외
2. 결측치 없는 '관리기관명', '제공기관코드', '제공기관명' 중에 적절한 것 고르기
3.  제공기관명, 관리기관명  '강남'으로 확인시  
제공기관명 : {'서울특별시 강남구청'} / 관리기관명 :  {'서울특별시 강남구청', '강원도 강릉시 강남동장'}  => '**강남구**'  로 변경해서 확인

4. 관리기관명보다는** 코드로 관리**되고 있는 제공기관으로 필터링 하는 것이 더 적합할 것으로 보인다.

5. 따라서,  **강남구의 제공기관 코드**를 이용하여 Filtering : '**3220000**'


In [43]:
gu_df = light_dataframe[light_dataframe.제공기관코드 == 3220000]
gu_df.head(5)
dong_df = gu_df[gu_df.보안등위치명.str.contains('역삼1동')]
dong_df.head(5)


,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명,Unnamed: 13
648502,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648503,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648504,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648505,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648506,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN


In [48]:
#결측치 일단 drop
#https://sacko.tistory.com/
#659540
dong_df = dong_df.dropna(subset = ['위도', '경도'])
dong_df.head(5)


,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명,Unnamed: 13
648502,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648503,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648504,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648505,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
648506,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN


In [129]:
df = dong_df.reset_index()
df.head(5)

,index,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명,Unnamed: 13
0,648502,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
1,648503,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
2,648504,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
3,648505,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
4,648506,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN


In [17]:
a = df[df.보안등위치명 == '역삼1동833']

In [18]:
len(a)

27

a.reset_index()

# 여기부터 반경 로직 시작
## 참고 사이트  :  
https://gaussian37.github.io/python-etc-%EC%9C%84%EB%8F%84,%EA%B2%BD%EB%8F%84-%EA%B0%84-%EA%B1%B0%EB%A6%AC/
https://codeday.me/ko/qa/20190404/234809.html

In [19]:
#!pip install haversine

In [63]:
from haversine import haversine

#출발지
strt = (37.506059, 127.036863) # (lat, lon)
#도착지
dstn = (37.509122, 127.043816)

#총 거리
d_m = haversine(strt, dstn, unit='m') # in meters
d_km = haversine(strt, dstn)  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m /5

print(dvd)

140.30702185064683


## 반경 내 모든 점 찾기

In [131]:
from  random import randint
df_ptInRange = pd.DataFrame(columns=['point_ID' , '위도' , '경도', '거리', '밝기', '경유여부'])
n = int(0)

for i in range(0, len(df)):
    
    #점과 출발점 거리측정 (strt는 이후에 경유지로 재설정 되어야한다.)
    dis = haversine(strt, [df.loc[i,'위도'] , df.loc[i,'경도'] ], unit='m')
    
    ## 1 block 내에 포함되는 points
    if dis <= dvd :
        tmp = {'point_ID' : 'Point_%d' % int(n)  , '위도' : df.loc[i,'위도'], '경도' : df.loc[i,'경도'] , '거리' : dis, '밝기' : randint(0,5) , '경유여부' : 0 } 
        df_ptInRange = df_ptInRange.append(df_ptInRange.from_dict(tmp, orient = 'index').T)
        n += 1
    
a = df_ptInRange.sort_values(by = ['거리','밝기'], ascending=[True, False] )
b = df_ptInRange.sort_values(by = ['밝기', '거리'], ascending=[False, True] )

a.head(20)

,point_ID,위도,경도,거리,밝기,경유여부
0,Point_26,37.5062,127.037,18.6378,3,0
0,Point_27,37.5059,127.037,19.8058,0,0
0,Point_28,37.506,127.037,28.1957,0,0
0,Point_58,37.5058,127.037,32.7353,5,0
0,Point_8,37.5063,127.037,40.0626,5,0
0,Point_48,37.5058,127.037,40.7487,4,0
0,Point_47,37.5058,127.037,40.7487,3,0
0,Point_29,37.5063,127.036,44.7265,3,0
0,Point_49,37.5061,127.036,46.116,0,0
0,Point_59,37.5056,127.037,49.3281,3,0


In [130]:
b.head(20)

,point_ID,위도,경도,거리,밝기,경유여부
0,Point_26,37.5062,127.037,18.6378,5,0
0,Point_27,37.5059,127.037,19.8058,5,0
0,Point_8,37.5063,127.037,40.0626,5,0
0,Point_47,37.5058,127.037,40.7487,5,0
0,Point_60,37.5059,127.037,58.9725,5,0
0,Point_44,37.5057,127.037,60.5065,5,0
0,Point_46,37.5056,127.036,77.0607,5,0
0,Point_55,37.5055,127.036,84.1844,5,0
0,Point_36,37.5057,127.035,130.849,5,0
0,Point_38,37.5054,127.036,134.507,5,0


In [108]:
df

,index,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명,Unnamed: 13
0,648502,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
1,648503,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
2,648504,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
3,648505,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
4,648506,역삼1동833,1,NaN,서울특별시 강남구 역삼1동833,37.493349,127.032033,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
5,648507,역삼1동832-25,1,NaN,서울특별시 강남구 역삼1동832-25,37.492834,127.031413,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
6,648508,역삼1동832-27,1,NaN,서울특별시 강남구 역삼1동832-27,37.492632,127.031508,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
7,648509,역삼1동832-28,1,NaN,서울특별시 강남구 역삼1동832-28,37.492480,127.031581,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
8,648510,역삼1동832-28,1,NaN,서울특별시 강남구 역삼1동832-28,37.492480,127.031581,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
9,648511,역삼1동832-29,1,NaN,서울특별시 강남구 역삼1동832-29,37.492343,127.031644,NaN,NaN,02-3423-6583,서울특별시 강남구청,2018-08-27,3220000,서울특별시 강남구,NaN
